In [ ]:
# Cell 1: Mount Google Drive to access data files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:

!pip install transformers[torch]
!pip install datasets
!pip install tqdm
!pip install seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=51215b1b4248fdf75dc7f96f13532bab882b033208f05e71900db4c96eda41a8
  S

In [ ]:


import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers")
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning, module="seqeval")
from transformers import RemBertTokenizerFast, RemBertForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
import torch
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
import gc
import os
from tqdm import tqdm
import re




In [ ]:
# Cell 3: Define a dictionary for grouping labels into broader categories
# Label Grouping dictionary
labels = {
    'B-Disease': 'B-Medical',
    'I-Disease': 'I-Medical',
    'B-Symptom': 'B-Medical',
    'I-Symptom': 'I-Medical',
    'B-AnatomicalStructure': 'B-Medical',
    'I-AnatomicalStructure': 'I-Medical',
    'B-MedicalProcedure': 'B-Medical',
    'I-MedicalProcedure': 'I-Medical',
    'B-Medication/Vaccine': 'B-Medical',
    'I-Medication/Vaccine': 'I-Medical',

    'B-OtherPROD': 'B-Product',
    'I-OtherPROD': 'I-Product',
    'B-Drink': 'B-Product',
    'I-Drink': 'I-Product',
    'B-Food': 'B-Product',
    'I-Food': 'I-Product',
    'B-Vehicle': 'B-Product',
    'I-Vehicle': 'I-Product',
    'B-Clothing': 'B-Product',
    'I-Clothing': 'I-Product',

    'B-OtherPER': 'B-Person',
    'I-OtherPER': 'I-Person',
    'B-SportsManager': 'B-Person',
    'I-SportsManager': 'I-Person',
    'B-Cleric': 'B-Person',
    'I-Cleric': 'I-Person',
    'B-Politician': 'B-Person',
    'I-Politician': 'I-Person',
    'B-Athlete': 'B-Person',
    'I-Athlete': 'I-Person',
    'B-Artist': 'B-Person',
    'I-Artist': 'I-Person',
    'B-Scientist': 'B-Person',
    'I-Scientist': 'I-Person',

    'B-MusicalGRP': 'B-Group',
    'I-MusicalGRP': 'I-Group',
    'B-PublicCorp': 'B-Group',
    'I-PublicCorp': 'I-Group',
    'B-PrivateCorp': 'B-Group',
    'I-PrivateCorp': 'I-Group',
    'B-AerospaceManufacturer': 'B-Group',
    'I-AerospaceManufacturer': 'I-Group',
    'B-SportsGRP': 'B-Group',
    'I-SportsGRP': 'I-Group',
    'B-CarManufacturer': 'B-Group',
    'I-CarManufacturer': 'I-Group',
    'B-ORG': 'B-Group',
    'I-ORG': 'I-Group',

    'B-VisualWork': 'B-CW',
    'I-VisualWork': 'I-CW',
    'B-MusicalWork': 'B-CW',
    'I-MusicalWork': 'I-CW',
    'B-WrittenWork': 'B-CW',
    'I-WrittenWork': 'I-CW',
    'B-ArtWork': 'B-CW',
    'I-ArtWork': 'I-CW',
    'B-Software': 'B-CW',
    'I-Software': 'I-CW',

    'B-Facility': 'B-Location',
    'I-Facility': 'I-Location',
    'B-OtherLOC': 'B-Location',
    'I-OtherLOC': 'I-Location',
    'B-HumanSettlement': 'B-Location',
    'I-HumanSettlement': 'I-Location',
    'B-Station': 'B-Location',
    'I-Station': 'I-Location',

    'O': 'O'
}


def convert_to_general_label(label):
    return labels.get(label)


Loading the perdiction files for 12 languages , individually. Due to High ram usage of the ram usage, this cannot be be done at single turn. the languages the ensemble model support are below.



In [ ]:
import os
import pickle
import numpy as np

#languages = ['EN-English', 'ES-Spanish', 'HI-Hindi', 'ZH-Chinese', 'SV-Swedish', 'FA-Farsi', 'FR-French', 'IT-Italian', 'PT-Portuguese', 'UK-Ukrainian', 'DE-German', 'BN-Bangla']
languages = ['DE-German']
predictions_base_path = '/content/drive/MyDrive/predictions'

# Create variables to store the predictions and label IDs for each language and model
predictions = {}
label_ids = {}

# Load the predictions and label IDs for each language
for lang in languages:
    # Load XLM-R predictions and label IDs
    print(f'Loading XLM-R predictions and label IDs for {lang}')
    xlmr_prediction_file_path = os.path.join(predictions_base_path, f'xlmr_{lang}_predictions.pkl')
    with open(xlmr_prediction_file_path, 'rb') as f:
        predictions[f'xlmr_{lang}'], label_ids[f'xlmr_{lang}'] = pickle.load(f)

    # Load mBERT predictions and label IDs
    print(f'Loading mBERT predictions and label IDs for {lang}')
    mbert_prediction_file_path = os.path.join(predictions_base_path, f'mbert_{lang}_predictions.pkl')
    with open(mbert_prediction_file_path, 'rb') as f:
        predictions[f'mbert_{lang}'], label_ids[f'mbert_{lang}'] = pickle.load(f)

    # Load RemBERT predictions and label IDs
    print(f'Loading RemBERT predictions and label IDs for {lang}')
    rembert_prediction_file_path = os.path.join(predictions_base_path, f'RemBert_{lang}_predictions.pkl')
    with open(rembert_prediction_file_path, 'rb') as f:
        predictions[f'rembert_{lang}'], label_ids[f'rembert_{lang}'] = pickle.load(f)


Loading XLM-R predictions and label IDs for DE-German
Loading mBERT predictions and label IDs for DE-German
Loading RemBERT predictions and label IDs for DE-German


To correct the problems with inconsistency with the lables while training, weightage is chosen as per evaluation on the test datasets for individual 36 models.For each models , the weights should be chosen in the range beetween **0.xx to 1.00**.

In [ ]:
max_seq_lengths = {}
for lang in languages:
    max_seq_lengths[lang] = max(predictions[f'xlmr_{lang}'].shape[1], predictions[f'mbert_{lang}'].shape[1], predictions[f'rembert_{lang}'].shape[1])

# Update max_sequence_length to be the maximum of all maximum sequence lengths
max_sequence_length = max(max_seq_lengths.values())
print(max_sequence_length)
# Define the weightage for each model ()NOte:the total weight should be 1
weights = {
    'xlmr': 0.2,  # Decreased weight for XLM-R
    'mbert':0.3,  # Increased weight for mBERT
    'rembert':0.5,  # Increased weight for RoBERTa-m
}


# Initialize an empty list to store the ensembled logits
ensemble_logits = []

# Iterate through the predictions for a specific language
for lang in languages:
    # Get the logits for each model
    xlmr_logits = predictions[f'xlmr_{lang}']
    mbert_logits = predictions[f'mbert_{lang}']
    rembert_logits = predictions[f'rembert_{lang}']

    # Ensure that all logits have the same shape (padding or truncating)
    common_shape = (xlmr_logits.shape[0], max_sequence_length, xlmr_logits.shape[2])
    xlmr_logits = np.pad(xlmr_logits, ((0, 0), (0, max_sequence_length - xlmr_logits.shape[1]), (0, 0)), mode='constant')
    mbert_logits = np.pad(mbert_logits, ((0, 0), (0, max_sequence_length - mbert_logits.shape[1]), (0, 0)), mode='constant')
    rembert_logits = np.pad(rembert_logits, ((0, 0), (0, max_sequence_length - rembert_logits.shape[1]), (0, 0)), mode='constant')

    # Calculate the weighted average of logits for each example
    weighted_average_logits = (
        weights['xlmr'] * xlmr_logits +
        weights['mbert'] * mbert_logits +
        weights['rembert'] * rembert_logits
    ) / (weights['xlmr'] + weights['mbert'] + weights['rembert'])

    # Append the ensembled logits to the list
    ensemble_logits.append(weighted_average_logits)

# Convert the ensemble_logits list to a NumPy array
ensemble_logits = np.array(ensemble_logits)

# Now you have the ensembled logits for each example in ensemble_logits

75


In [ ]:
if lang in languages:
    # Retrieve the label_ids for the RemBERT model for the specified language
    rembert_label_ids = label_ids[f'rembert_{lang}']
    print(f'Label IDs for RemBERT model for {lang}:')

else:
    print(f'{lang} is not found in the languages list.')


Label IDs for RemBERT model for DE-German:


In [ ]:
rembert_index_to_label_base_path = '/content/drive/MyDrive/RemBert_index_to_label'

rembert_index_to_label = {}
for lang in languages:
    # Create the RemBERT index_to_label directory for this language
    rembert_index_to_label_dir = f'{rembert_index_to_label_base_path}/{lang}'
    if not os.path.exists(rembert_index_to_label_dir):
        os.makedirs(rembert_index_to_label_dir)

# Initialize an empty dictionary for label_to_index mapping
label_to_index = {}

# Iterate through the existing index_to_label mapping
for index, label in rembert_index_to_label.items():
    # Assign the label as the key and the index as the value
    label_to_index[label] = index


Master Labelling consists three options. Choose from the List below:

**rembert**,
**xlmr**,
**mbert**

In [ ]:
'''master_label_mapping={'xlmr': {'I-Cleric': 0, 'I-Artist': 1, 'B-AerospaceManufacturer': 2, 'B-Artist': 3, 'B-MusicalWork': 4, 'I-Disease': 5, 'I-Software': 6, 'I-PrivateCorp': 7, 'B-OtherLOC': 8, 'I-AerospaceManufacturer': 9, 'I-HumanSettlement': 10, 'B-Clothing': 11, 'I-Facility': 12, 'I-VisualWork': 13, 'I-Station': 14, 'I-OtherPROD': 15, 'B-MedicalProcedure': 16, 'I-MusicalWork': 17, 'I-AnatomicalStructure': 18, 'B-ArtWork': 19, 'B-Athlete': 20, 'B-Station': 21, 'B-Food': 22, 'B-AnatomicalStructure': 23, 'B-OtherPER': 24, 'B-HumanSettlement': 25, 'I-OtherLOC': 26, 'B-Scientist': 27, 'I-Politician': 28, 'B-CarManufacturer': 29, 'B-OtherPROD': 30, 'I-Food': 31, 'B-SportsManager': 32, 'B-Cleric': 33, 'I-WrittenWork': 34, 'I-Clothing': 35, 'B-PrivateCorp': 36, 'B-Politician': 37, 'B-Facility': 38, 'I-SportsManager': 39, 'B-PublicCorp': 40, 'I-PublicCorp': 41, 'I-Symptom': 42, 'I-Scientist': 43, 'B-Software': 44, 'B-Vehicle': 45, 'I-Medication/Vaccine': 46, 'I-ORG': 47, 'B-MusicalGRP': 48, 'I-MusicalGRP': 49, 'I-Drink': 50, 'B-ORG': 51, 'I-SportsGRP': 52, 'B-VisualWork': 53, 'I-Athlete': 54, 'B-Medication/Vaccine': 55, 'B-Symptom': 56, 'I-MedicalProcedure': 57, 'B-Disease': 58, 'I-OtherPER': 59, 'I-ArtWork': 60, 'B-WrittenWork': 61, 'B-SportsGRP': 62, 'I-CarManufacturer': 63, 'I-Vehicle': 64, 'O': 65, 'B-Drink': 66}, 'mbert': {'I-CarManufacturer': 0, 'I-Disease': 1, 'B-Athlete': 2, 'B-Station': 3, 'I-AerospaceManufacturer': 4, 'I-Vehicle': 5, 'I-Symptom': 6, 'B-MusicalGRP': 7, 'I-HumanSettlement': 8, 'I-MusicalWork': 9, 'I-Facility': 10, 'B-SportsGRP': 11, 'B-Food': 12, 'I-PrivateCorp': 13, 'I-Artist': 14, 'B-WrittenWork': 15, 'I-SportsManager': 16, 'B-Software': 17, 'B-PublicCorp': 18, 'O': 19, 'B-Disease': 20, 'B-Artist': 21, 'I-Cleric': 22, 'B-OtherPROD': 23, 'I-Medication/Vaccine': 24, 'I-MusicalGRP': 25, 'I-Software': 26, 'I-Drink': 27, 'I-OtherPER': 28, 'B-CarManufacturer': 29, 'I-Clothing': 30, 'B-Clothing': 31, 'B-MedicalProcedure': 32, 'I-OtherLOC': 33, 'B-Symptom': 34, 'B-Vehicle': 35, 'B-ArtWork': 36, 'B-ORG': 37, 'I-Athlete': 38, 'I-OtherPROD': 39, 'B-VisualWork': 40, 'B-Drink': 41, 'I-AnatomicalStructure': 42, 'B-Scientist': 43, 'I-WrittenWork': 44, 'B-Facility': 45, 'B-Cleric': 46, 'B-Politician': 47, 'B-PrivateCorp': 48, 'B-SportsManager': 49, 'I-MedicalProcedure': 50, 'B-HumanSettlement': 51, 'I-Scientist': 52, 'I-Station': 53, 'I-SportsGRP': 54, 'B-Medication/Vaccine': 55, 'B-AnatomicalStructure': 56, 'B-OtherLOC': 57, 'B-MusicalWork': 58, 'I-ORG': 59, 'I-VisualWork': 60, 'I-Food': 61, 'I-Politician': 62, 'I-PublicCorp': 63, 'B-AerospaceManufacturer': 64, 'I-ArtWork': 65, 'B-OtherPER': 66}, 'rembert': {'I-MusicalGRP': 0, 'I-PrivateCorp': 1, 'I-SportsGRP': 2, 'B-Station': 3, 'B-Drink': 4, 'I-Artist': 5, 'B-Athlete': 6, 'I-Software': 7, 'I-Vehicle': 8, 'B-CarManufacturer': 9, 'I-AnatomicalStructure': 10, 'B-OtherPROD': 11, 'O': 12, 'B-MusicalGRP': 13, 'B-PublicCorp': 14, 'I-CarManufacturer': 15, 'I-WrittenWork': 16, 'B-OtherLOC': 17, 'I-Politician': 18, 'I-Symptom': 19, 'B-Scientist': 20, 'I-Medication/Vaccine': 21, 'B-Food': 22, 'B-SportsManager': 23, 'B-AnatomicalStructure': 24, 'I-Station': 25, 'B-ArtWork': 26, 'I-ArtWork': 27, 'B-PrivateCorp': 28, 'I-AerospaceManufacturer': 29, 'B-MedicalProcedure': 30, 'B-Artist': 31, 'I-PublicCorp': 32, 'I-OtherLOC': 33, 'I-MedicalProcedure': 34, 'I-OtherPER': 35, 'B-Facility': 36, 'I-HumanSettlement': 37, 'I-OtherPROD': 38, 'B-Clothing': 39, 'I-VisualWork': 40, 'B-VisualWork': 41, 'B-Software': 42, 'B-Disease': 43, 'I-ORG': 44, 'B-SportsGRP': 45, 'I-Food': 46, 'I-SportsManager': 47, 'I-Clothing': 48, 'B-MusicalWork': 49, 'B-AerospaceManufacturer': 50, 'I-Cleric': 51, 'B-Medication/Vaccine': 52, 'B-HumanSettlement': 53, 'B-Politician': 54, 'B-Vehicle': 55, 'I-Athlete': 56, 'B-ORG': 57, 'B-Symptom': 58, 'I-Facility': 59, 'I-MusicalWork': 60, 'I-Disease': 61, 'B-WrittenWork': 62, 'B-OtherPER': 63, 'I-Drink': 64, 'I-Scientist': 65, 'B-Cleric': 66}}
label_mapping = master_label_mapping['rembert']

# Create the index_to_label dictionary for the RemBERT model
index_to_label = {i: label for label, i in label_mapping.items()}
'''
master_label_mapping={
  'I-CarManufacturer': 0,
  'I-Disease': 1,
  'B-Athlete': 2,
  'B-Station': 3,
  'I-AerospaceManufacturer': 4,
  'I-Vehicle': 5,
  'I-Symptom': 6,
  'B-MusicalGRP': 7,
  'I-HumanSettlement': 8,
  'I-MusicalWork': 9,
  'I-Facility': 10,
  'B-SportsGRP': 11,
  'B-Food': 12,
  'I-PrivateCorp': 13,
  'I-Artist': 14,
  'B-WrittenWork': 15,
  'I-SportsManager': 16,
  'B-Software': 17,
  'B-PublicCorp': 18,
  'O': 19,
  'B-Disease': 20,
  'B-Artist': 21,
  'I-Cleric': 22,
  'B-OtherPROD': 23,
  'I-Medication/Vaccine': 24,
  'I-MusicalGRP': 25,
  'I-Software': 26,
  'I-Drink': 27,
  'I-OtherPER': 28,
  'B-CarManufacturer': 29,
  'I-Clothing': 30,
  'B-Clothing': 31,
  'B-MedicalProcedure': 32,
  'I-OtherLOC': 33,
  'B-Symptom': 34,
  'B-Vehicle': 35,
  'B-ArtWork': 36,
  'B-ORG': 37,
  'I-Athlete': 38,
  'I-OtherPROD': 39,
  'B-VisualWork': 40,
  'B-Drink': 41,
  'I-AnatomicalStructure': 42,
  'B-Scientist': 43,
  'I-WrittenWork': 44,
  'B-Facility': 45,
  'B-Cleric': 46,
  'B-Politician': 47,
  'B-PrivateCorp': 48,
  'B-SportsManager': 49,
  'I-MedicalProcedure': 50,
  'B-HumanSettlement': 51,
  'I-Scientist': 52,
  'I-Station': 53,
  'I-SportsGRP': 54,
  'B-Medication/Vaccine': 55,
  'B-AnatomicalStructure': 56,
  'B-OtherLOC': 57,
  'B-MusicalWork': 58,
  'I-ORG': 59,
  'I-VisualWork': 60,
  'I-Food': 61,
  'I-Politician': 62,
  'I-PublicCorp': 63,
  'B-AerospaceManufacturer': 64,
  'I-ArtWork': 65,
  'B-OtherPER': 66
}
index_to_label = {i: label for label, i in master_label_mapping.items()}

In [ ]:

def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=-1)
    true_labels = p.label_ids
    true_label_list = [[] for _ in range(true_labels.shape[0])]
    pred_label_list = [[] for _ in range(true_labels.shape[0])]

    for i in range(true_labels.shape[0]):
        for j in range(true_labels.shape[1]):
            if true_labels[i, j] != -100:
                true_label_index = int(true_labels[i, j])
                pred_label_index = int(predictions[0, i, j])
                true_label = index_to_label.get(true_label_index, 'UNKNOWN_LABEL')
                pred_label = index_to_label.get(pred_label_index, 'UNKNOWN_LABEL')
                true_label_list[i].append(true_label)
                pred_label_list[i].append(pred_label)

        true_label_list[i] = [convert_to_general_label(label) for label in true_label_list[i]]
        pred_label_list[i] = [convert_to_general_label(label) for label in pred_label_list[i]]

    results = {
        "precision": precision_score(true_label_list, pred_label_list),
        "recall": recall_score(true_label_list, pred_label_list),
        "f1": f1_score(true_label_list, pred_label_list),
    }

    report = classification_report(true_label_list, pred_label_list)
    print(report)

    return results



In [ ]:
# Create a new object to hold the predictions and label IDs
#print(ensemble_logits.shape)
print(f'results for {lang}')
p = type('', (), {})()
p.predictions = ensemble_logits
p.label_ids = rembert_label_ids

# Call the compute_metrics function with the new object as input
# Call the compute_metrics function with the new object and rembert_label_mapping as input
# Get the label mapping for the RemBERT model


results = compute_metrics(p)

# Print the results
print(f'Precision: {results["precision"]}')
print(f'Recall: {results["recall"]}')
print(f'F1 Score: {results["f1"]}')




results for DE-German
              precision    recall  f1-score   support

          CW       0.98      0.97      0.98      7450
       Group       0.93      0.92      0.93     10068
    Location       0.98      0.98      0.98      7165
     Medical       0.94      0.92      0.93      7274
      Person       0.97      0.95      0.96      6577
     Product       0.99      1.00      1.00    239965

   micro avg       0.99      0.99      0.99    278499
   macro avg       0.97      0.96      0.96    278499
weighted avg       0.99      0.99      0.99    278499

Precision: 0.989595080919167
Recall: 0.9893356888175541
F1 Score: 0.9894653678682048



# RESULTS:
#1.English
results for EN-English
              precision    recall  f1-score   support

          CW       0.92      0.89      0.91     87937
       Group       0.92      0.92      0.92    565500
    Location       0.91      0.88      0.90     62901
     Medical       0.89      0.88      0.89     87504
      Person       0.94      0.93      0.94    232998
     Product       0.85      0.86      0.85    131721

   micro avg       0.91      0.91      0.91   1168561
   macro avg       0.91      0.90      0.90   1168561
weighted avg       0.91      0.91      0.91   1168561

Precision: 0.9146125065338799
Recall: 0.9103914986038384
F1 Score: 0.9124971212349291

#2.Spanish
results for ES-Spanish
              precision    recall  f1-score   support

          CW       0.92      0.94      0.93    116808
       Group       0.93      0.92      0.92    127220
    Location       1.00      1.00      1.00   3242720
     Medical       0.93      0.90      0.92     61915
      Person       0.94      0.92      0.93    104692
     Product       0.93      0.93      0.93    154782

   micro avg       0.99      0.99      0.99   3808137
   macro avg       0.94      0.93      0.94   3808137
weighted avg       0.99      0.99      0.99   3808137

Precision: 0.9858252670364994
Recall: 0.9859836975402934
F1 Score: 0.9859044759236254
#3.Hindi
results for HI-Hindi
              precision    recall  f1-score   support

          CW       0.99      0.99      0.99      7917
       Group       0.99      0.99      0.99      7947
    Location       0.99      0.99      0.99      4103
     Medical       0.98      0.98      0.98      5005
      Person       0.99      0.99      0.99     36964
     Product       0.99      0.99      0.99      8342

   micro avg       0.99      0.99      0.99     70278
   macro avg       0.99      0.99      0.99     70278
weighted avg       0.99      0.99      0.99     70278

Precision: 0.9905342120621183
Recall: 0.9901818492273542
F1 Score: 0.9903579993026449
#4.Bangla
results for BN-Bangla
              precision    recall  f1-score   support

          CW       0.98      0.98      0.98      5313
       Group       0.97      0.96      0.97      8089
    Location       0.99      0.99      0.99      8808
     Medical       0.97      0.96      0.96      4598
      Person       0.98      0.98      0.98      5896
     Product       1.00      1.00      1.00    215260

   micro avg       1.00      1.00      1.00    247964
   macro avg       0.98      0.98      0.98    247964
weighted avg       1.00      1.00      1.00    247964

Precision: 0.9952894678469884
Recall: 0.9952533432272427
F1 Score: 0.9952714052093184
#5.Chinese
results for ZH-Chinese
              precision    recall  f1-score   support

          CW       0.95      0.95      0.95     11413
       Group       0.94      0.93      0.94     29888
    Location       0.96      0.94      0.95     10442
     Medical       0.92      0.93      0.93     16904
      Person       0.95      0.94      0.95     13149
     Product       0.99      0.99      0.99    357222

   micro avg       0.98      0.98      0.98    439018
   macro avg       0.95      0.95      0.95    439018
weighted avg       0.98      0.98      0.98    439018

Precision: 0.9822869316563565
Recall: 0.9819893489560793
F1 Score: 0.9821381177647188
#6.Swedish
results for SV-Swedish
              precision    recall  f1-score   support

          CW       0.94      0.93      0.93    129935
       Group       0.87      0.90      0.89    113751
    Location       0.97      0.97      0.97     46131
     Medical       0.93      0.90      0.91     94813
      Person       0.94      0.92      0.93     65040
     Product       0.99      1.00      1.00   2417877

   micro avg       0.98      0.98      0.98   2867547
   macro avg       0.94      0.94      0.94   2867547
weighted avg       0.98      0.98      0.98   2867547

Precision: 0.9835651665040716
Recall: 0.9840940706464445
F1 Score: 0.9838295474908989
#7.Farsi
results for FA-Farsi
              precision    recall  f1-score   support

          CW       0.91      0.92      0.92    108372
       Group       0.92      0.90      0.91    106740
    Location       0.99      0.99      0.99   3288013
     Medical       0.94      0.93      0.93     55366
      Person       0.93      0.90      0.92     75750
     Product       0.91      0.91      0.91    146611

   micro avg       0.98      0.98      0.98   3780852
   macro avg       0.93      0.93      0.93   3780852
weighted avg       0.98      0.98      0.98   3780852

Precision: 0.981371741945618
Recall: 0.9815433135176939
F1 Score: 0.981457520233419
#8.French
results for FR-French
              precision    recall  f1-score   support

          CW       0.92      0.92      0.92    189956
       Group       0.89      0.89      0.89    177572
    Location       0.93      0.90      0.92     37854
     Medical       0.90      0.88      0.89    117354
      Person       0.91      0.88      0.90     94118
     Product       0.99      0.99      0.99   2984667

   micro avg       0.98      0.98      0.98   3601521
   macro avg       0.92      0.91      0.92   3601521
weighted avg       0.98      0.98      0.98   3601521

Precision: 0.9764103101866801
Recall: 0.9772634950622251
F1 Score: 0.9768367163281452
#9.Italian
results for IT-Italian
              precision    recall  f1-score   support

          CW       0.87      0.89      0.88     77672
       Group       0.93      0.91      0.92     96278
    Location       0.99      0.99      0.99   2749549
     Medical       0.95      0.96      0.96    197768
      Person       0.97      0.97      0.97    256915
     Product       0.92      0.89      0.91     83167

   micro avg       0.98      0.98      0.98   3461349
   macro avg       0.94      0.94      0.94   3461349
weighted avg       0.98      0.98      0.98   3461349

Precision: 0.9822461658603305
Recall: 0.9825943584423299
F1 Score: 0.9824202312994436
#10.Portugese
results for PT-Portuguese
              precision    recall  f1-score   support

          CW       0.88      0.91      0.89     79937
       Group       0.93      0.90      0.92     79008
    Location       0.99      0.99      0.99   2948519
     Medical       0.95      0.95      0.95    162371
      Person       0.95      0.94      0.94    151038
     Product       0.94      0.90      0.92     87677

   micro avg       0.98      0.98      0.98   3508550
   macro avg       0.94      0.93      0.94   3508550
weighted avg       0.98      0.98      0.98   3508550

Precision: 0.983299758494776
Recall: 0.9837254706360177
F1 Score: 0.9835125684981618
#11.Ukranian
results for UK-Ukrainian
              precision    recall  f1-score   support

          CW       0.87      0.90      0.89     63992
       Group       0.92      0.88      0.90     65932
    Location       0.99      0.99      0.99   2722230
     Medical       0.95      0.94      0.94    137036
      Person       0.93      0.92      0.92     98315
     Product       0.94      0.90      0.92     85602

   micro avg       0.98      0.98      0.98   3173107
   macro avg       0.93      0.92      0.93   3173107
weighted avg       0.98      0.98      0.98   3173107

Precision: 0.9828441900135161
Recall: 0.9828943682012614
F1 Score: 0.9828692784669549
#12.German
results for DE-German
              precision    recall  f1-score   support

          CW       0.98      0.97      0.98      7450
       Group       0.93      0.92      0.93     10068
    Location       0.98      0.98      0.98      7165
     Medical       0.94      0.92      0.93      7274
      Person       0.97      0.95      0.96      6577
     Product       0.99      1.00      1.00    239965

   micro avg       0.99      0.99      0.99    278499
   macro avg       0.97      0.96      0.96    278499
weighted avg       0.99      0.99      0.99    278499

Precision: 0.989595080919167
Recall: 0.9893356888175541
F1 Score: 0.9894653678682048

In [ ]:
code to break

SyntaxError: ignored

# **Code for saving the Index_to Lable**




In [ ]:
import pickle

# Set the base paths for the tokenizers
rembert_tokenizer_base_path = '/content/drive/MyDrive/RemBert_fin_models'
mbert_tokenizer_base_path = '/content/drive/MyDrive/mbert_fin_models'
xlm_tokenizer_base_path = '/content/drive/MyDrive/xlm_fin_models'

# Define the list of languages
languages = ['EN-English', 'ES-Spanish', 'HI-Hindi', 'ZH-Chinese', 'SV-Swedish', 'FA-Farsi', 'FR-French', 'IT-Italian', 'PT-Portuguese', 'UK-Ukrainian', 'DE-German', 'BN-Bangla']

# Load the tokenizers for all languages
rembert_tokenizers = {}
mbert_tokenizers = {}
xlm_tokenizers = {}
for lang in languages:
    # Load the RemBERT tokenizer for this language
    rembert_tokenizer_path = f'{rembert_tokenizer_base_path}/{lang}/tokenizer/tokenizer.pkl'
    with open(rembert_tokenizer_path, 'rb') as f:
        rembert_tokenizers[lang] = pickle.load(f)

    # Load the mBERT tokenizer for this language
    mbert_tokenizer_path = f'{mbert_tokenizer_base_path}/{lang}/tokenizer/tokenizer.pkl'
    with open(mbert_tokenizer_path, 'rb') as f:
        mbert_tokenizers[lang] = pickle.load(f)

    # Load the XLM tokenizer for this language
    xlm_tokenizer_path = f'{xlm_tokenizer_base_path}/{lang}/tokenizer/tokenizer.pkl'
    with open(xlm_tokenizer_path, 'rb') as f:
        xlm_tokenizers[lang] = pickle.load(f)


In [ ]:
# Define dictionaries to store the index_to_label mappings for all models and languages
rembert_index_to_label = {}
mbert_index_to_label = {}
xlm_index_to_label = {}

for lang in languages:
    # Get the RemBERT tokenizer for this language
    rembert_tokenizer = rembert_tokenizers[lang]

    # Get the vocabulary from the RemBERT tokenizer
    rembert_vocab = rembert_tokenizer.get_vocab()

    # Create a dictionary that maps indices to tokens for the RemBERT tokenizer
    rembert_index_to_label[lang] = {index: token for token, index in rembert_vocab.items()}

    # Get the mBERT tokenizer for this language
    mbert_tokenizer = mbert_tokenizers[lang]

    # Get the vocabulary from the mBERT tokenizer
    mbert_vocab = mbert_tokenizer.get_vocab()

    # Create a dictionary that maps indices to tokens for the mBERT tokenizer
    mbert_index_to_label[lang] = {index: token for token, index in mbert_vocab.items()}

    # Get the XLM tokenizer for this language
    xlm_tokenizer = xlm_tokenizers[lang]

    # Get the vocabulary from the XLM tokenizer
    xlm_vocab = xlm_tokenizer.get_vocab()

    # Create a dictionary that maps indices to tokens for the XLM tokenizer
    xlm_index_to_label[lang] = {index: token for token, index in xlm_vocab.items()}


In [ ]:
import os

# Set the base paths for the index_to_label data on Google Drive
rembert_index_to_label_base_path = '/content/drive/MyDrive/RemBert_index_to_label'
mbert_index_to_label_base_path = '/content/drive/MyDrive/mbert_index_to_label'
xlm_index_to_label_base_path = '/content/drive/MyDrive/xlm_index_to_label'

# Define the list of languages
languages = ['EN-English', 'ES-Spanish', 'HI-Hindi', 'ZH-Chinese', 'SV-Swedish', 'FA-Farsi', 'FR-French', 'IT-Italian', 'PT-Portuguese', 'UK-Ukrainian', 'DE-German', 'BN-Bangla']

# Create the directories for all models and languages if they don't already exist
for lang in languages:
    # Create the RemBERT index_to_label directory for this language
    rembert_index_to_label_dir = f'{rembert_index_to_label_base_path}/{lang}'
    if not os.path.exists(rembert_index_to_label_dir):
        os.makedirs(rembert_index_to_label_dir)

    # Create the mBERT index_to_label directory for this language
    mbert_index_to_label_dir = f'{mbert_index_to_label_base_path}/{lang}'
    if not os.path.exists(mbert_index_to_label_dir):
        os.makedirs(mbert_index_to_label_dir)

    # Create the XLM index_to_label directory for this language
    xlm_index_to_label_dir = f'{xlm_index_to_label_base_path}/{lang}'
    if not os.path.exists(xlm_index_to_label_dir):
        os.makedirs(xlm_index_to_label_dir)


# Save the index_to_label data for all models and languages
for lang in languages:
    # Save the RemBERT index_to_label data for this language
    rembert_index_to_label_path = f'{rembert_index_to_label_base_path}/{lang}/index_to_label.pkl'
    with open(rembert_index_to_label_path, 'wb') as f:
        pickle.dump(rembert_index_to_label[lang], f)

    # Save the mBERT index_to_label data for this language
    mbert_index_to_label_path = f'{mbert_index_to_label_base_path}/{lang}/index_to_label.pkl'
    with open(mbert_index_to_label_path, 'wb') as f:
        pickle.dump(mbert_index_to_label[lang], f)

    # Save the XLM index_to_label data for this language
    xlm_index_to_label_path = f'{xlm_index_to_label_base_path}/{lang}/index_to_label.pkl'
    with open(xlm_index_to_label_path, 'wb') as f:
        pickle.dump(xlm_index_to_label[lang], f)
